In [11]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path

TICKER = "SPY"  # adjust if needed
possible_paths = [Path(f"../options_{TICKER}.csv"), Path(f"../../options_{TICKER}.csv"), Path(f"options_{TICKER}.csv")]
for candidate in possible_paths:
    if candidate.exists():
        CSV_PATH = candidate
        break
else:
    raise FileNotFoundError(f"Could not locate options_{TICKER}.csv near this notebook.")

df = pd.read_csv(CSV_PATH)
required_cols = {"S0", "K", "T", "iv"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"CSV is missing required columns: {missing}")

df = df.dropna(subset=["S0", "K", "T", "iv", "type"]).copy()
df = df[df['type'].astype(str).str.upper() == 'C']
df = df[(df["K"] > 0) & (df["T"] > 0) & (df["iv"] > 0)]
if df.empty:
    raise ValueError("No call rows remain after filtering.")

spot_ref = df['S0'].median()
strike_min = spot_ref - 150.0
strike_max = spot_ref + 150.0
T_min, T_max = df['T'].min(), df['T'].max()
max_strikes = min(40, df["K"].nunique())
max_maturities = min(40, df["T"].nunique())
strike_vals = np.linspace(strike_min, strike_max, max_strikes)
maturity_vals = np.linspace(T_min, T_max, max_maturities)
subset = df[(df['K'] >= strike_min) & (df['K'] <= strike_max)].copy()
if subset.empty:
    raise ValueError('No strikes within ±150 of the spot price.')
pivot = subset.pivot_table(index="T", columns="K", values="iv", aggfunc="mean")
pivot = pivot.dropna(how="all").dropna(axis=1, how="all")
if pivot.empty:
    raise ValueError("Not enough strike/maturity combinations to build the surface.")
pivot = pivot.apply(lambda col: col.fillna(col.mean()))
pivot = pivot.fillna(pivot.mean().mean())

T_vals = pivot.index.values
K_vals = pivot.columns.values
Vols = pivot.values
K_mesh, T_mesh = np.meshgrid(K_vals, T_vals)

fig = go.Figure(data=[
    go.Surface(x=K_mesh, y=T_mesh, z=Vols, colorscale="Viridis", colorbar=dict(title="iv"))
])
fig.update_layout(
    title=f"Call Implied Vol Surface ({TICKER})",
    scene=dict(
        xaxis_title="Strike K",
        yaxis_title="Maturity T (yrs)",
        zaxis_title="Implied Volatility",
        xaxis=dict(range=[strike_min, strike_max]),
        yaxis=dict(range=[T_min, T_max])
    ),
    template="plotly_dark",
    width=900,
    height=600,
)
fig.show()
